In [5]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt

In [6]:
def getWeights_FFD(d,thres):
    w,k=[1.],1
    while True:
        w_=-w[-1]/k*(d-k+1)
        if abs(w_)<thres:break
        w.append(w_);k+=1
    return np.array(w[::-1]).reshape(-1,1)

def fracDiff_FFD(series,d,thres=1e-5):
    # Constant width window (new solution)
    w=getWeights_FFD(d,thres)
    df={}
    for name in series.columns:
        df_= pd.Series(series[name].values,index=series.index).fillna(method='ffill').dropna()
        x = pd.Series(0,index=df_.index)
        for k in range(w.shape[0]):
            x = x+w[k,0]*df_.shift(-k)
    #df[name]=x.dropna().copy(deep=True)
    df[name]=x.shift(k).copy(deep=True)
    df=pd.concat(df,axis=1)
    return df

def findMinD(series):
    for d in np.linspace(0,1,11):
        df_=fracDiff_FFD(series,d,thres=0.01).dropna()
        res=adfuller(df_.iloc[:,0].values,maxlag=1,regression='c',autolag=None)
        if (res[0]<=res[4]['5%']):
            return d
    return 1.0

In [3]:
monthlydf = pd.read_excel('./PredictorData2019.xlsx', sheet_name='Monthly')
quarterlydf = pd.read_excel('./PredictorData2019.xlsx', sheet_name='Quarterly')
annualdf = pd.read_excel('./PredictorData2019.xlsx', sheet_name='Annual')   

c:\Users\tkkim\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\Users\tkkim\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\Users\tkkim\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [11]:
annualdf.columns

Index(['yyyy', 'Index', 'D12', 'E12', 'b/m', 'tbl', 'AAA', 'BAA', 'lty', 'cay',
       'ntis', 'Rfree', 'infl', 'eqis', 'ltr', 'corpr', 'svar', 'csp', 'ik',
       'CRSP_SPvw', 'CRSP_SPvwx'],
      dtype='object')

In [12]:
monthlydf.columns

Index(['yyyymm', 'Index', 'D12', 'E12', 'b/m', 'tbl', 'AAA', 'BAA', 'lty',
       'ntis', 'Rfree', 'infl', 'ltr', 'corpr', 'svar', 'csp', 'CRSP_SPvw',
       'CRSP_SPvwx'],
      dtype='object')

In [4]:
monthlydf.head()

,yyyymm,Index,D12,E12,b/m,tbl,AAA,BAA,lty,ntis,Rfree,infl,ltr,corpr,svar,csp,CRSP_SPvw,CRSP_SPvwx
0,187101,4.44,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004967,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,187102,4.50,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004525,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,187103,4.61,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004252,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,187104,4.74,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004643,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,187105,4.86,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.003698,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#dividend price ratio
monthlydf['dp'] = np.log(monthlydf['D12']) - np.log(monthlydf['Index'])

#dividend yield
monthlydf['dy'] = np.log(monthlydf['D12']) - np.log(monthlydf['Index'].shift(-1))

#earnings price ratio
monthlydf['ep'] = np.log(monthlydf['E12']) - np.log(monthlydf['Index'])

#dividend payout ratio
monthlydf['de'] = np.log(monthlydf['D12']) - np.log(monthlydf['E12'])

#default yield spread
monthlydf['dfy'] = monthlydf['BAA'] - monthlydf['AAA']

#term spread
monthlydf['tms'] = monthlydf['lty'] - monthlydf['tbl']

#default return spread
monthlydf['dfr'] = monthlydf['corpr'] - monthlydf['lty']




0      -2.837728
1      -2.851151
2      -2.875302
3      -2.903111
4      -2.928112
          ...   
1783   -3.941330
1784   -3.951689
1785   -3.965984
1786   -3.993568
1787   -4.015896
Length: 1788, dtype: float64